# Fairness Post-Processing: Adult Income (Sex as Protected Attribute)

In this notebook we switch gears from race to **sex** as our protected attribute.  
We will:

1. Load & pre-process data (sensitive = **sex**)

2. Build a Random-Forest baseline

3. Apply three post-processing methods:  
   - **Origin** (no repair)  
   - **Barycentre** (OT-based full repair)  
   - **Partial** (tunable, softer repair)  

4. Compare Disparate Impact, F1 variants, and TV distance  

5. Compute feature-importance and draw conclusions

## 1  Imports & basic setup

In [1]:
import sys
import os
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from aif360.datasets import BinaryLabelDataset, AdultDataset

# TODO: change the import method
repo_root = os.path.dirname(os.getcwd())
sys.path.insert(0, repo_root)
repair_folder = os.path.join(repo_root, "humancompatible", "repair")
sys.path.insert(0, repair_folder)
from humancompatible.repair.cost import *
from humancompatible.repair.coupling_utils import *
from humancompatible.repair.data_analysis import *
from humancompatible.repair.group_blind_repair import *
from humancompatible.repair.metrics import *

from humancompatible.repair.roc_postprocess import ROCpostprocess
from humancompatible.repair.proj_postprocess import Projpostprocess

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


We silence FutureWarnings so you’ll only see the key outputs below.

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 2  Utility helpers

Here are three functions:

- **`load_data`** - merges train/test, encodes `S` & `Y`, bins skewed numerics  

- **`categerise`** - simple numeric binning for one column  

- **`choose_x`** - computes per-feature Total-Variation to shortlist the most imbalanced axes  

In [3]:
def load_data(data_path,var_list,pa):
    column_names = ['age', 'workclass', 'fnlwgt', 'education',
                'education-num', 'marital-status', 'occupation', 'relationship',
                'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
                'native-country', 'Y']
    na_values=['?']
    pa_dict={'Male':1,'Female':0,'White':1,'Black':0}
    label_dict={'>50K.':1,'>50K':1,'<=50K.':0,'<=50K':0}
    train_path = os.path.join(data_path, 'adult.data')
    test_path = os.path.join(data_path, 'adult.test')
    train = pd.read_csv(train_path, header=None,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    test = pd.read_csv(test_path, header=0,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    messydata = pd.concat([test, train], ignore_index=True)[var_list+[pa,'Y']]
    messydata=messydata.rename(columns={pa:'S'})
    messydata['S']=messydata['S'].replace(pa_dict)
    messydata['Y']=messydata['Y'].replace(label_dict)
    messydata=messydata[(messydata['S']==0)|(messydata['S']==1)]
    for col in var_list+['S','Y']:
        messydata[col]=messydata[col].astype('int64')
    messydata['W']=1
    bins_capitalgain=[100,3500,7500,10000]
    bins_capitalloss=[100,1600,1900,2200]
    bins_age=[26,36,46,56]
    bins_hours=[21,36,46,61]

    messydata=categerise(messydata,'age',bins_age)
    # messydata=categerise(messydata,'hours-per-week',bins_hours)
    messydata=categerise(messydata,'capital-gain',bins_capitalgain)
    messydata=categerise(messydata,'capital-loss',bins_capitalloss)
    
    return messydata

def categerise(df,col,bins):
    for i in range(len(bins)+1):
        if i == 0:
            df.loc[df[col] < bins[i], col] = i
        elif i == len(bins):
            df.loc[df[col] >= bins[i-1], col] = i
        else:
            df.loc[(df[col] >= bins[i-1])& (df[col] < bins[i]), col] = i        
    return df

def choose_x(var_list,messydata):
    tv_dist=dict()
    for x_name in var_list:
        x_range_single=list(pd.pivot_table(messydata,index=x_name,values=['W'])[('W')].index) 
        dist=rdata_analysis(messydata,x_range_single,x_name)
        tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
    x_list=[]
    for key,val in tv_dist.items():
        if val>0.1:
            x_list+=[key]  
    return x_list,tv_dist

### Protected-attribute setup

- **Protected attribute**: `pa = "sex"`

- **Privileged** = Male (`S = 1`), **Unprivileged** = Female (`S = 0`)  

- We tune our threshold grid around 0.05, since sex-based gaps tend to be subtler than race.

In [4]:
data_path='..//data//adult'
var_list=['age','capital-gain','capital-loss','education-num'] #'hours-per-week',
pa='sex'
favorable_label = 1
var_dim=len(var_list)

K=200
e=0.01

if pa == 'sex':
    thresh=0.05
elif pa == 'race':
    thresh=0.1

messydata = load_data(data_path,var_list,pa)
x_list,tv_dist = choose_x(var_list,messydata)

X=messydata[var_list+['S','W']].to_numpy() # [X,S,W]
y=messydata['Y'].to_numpy() #[Y]

In [5]:
tv_dist

{'age': np.float64(0.1010227688866829),
 'capital-gain': np.float64(0.036924675713792855),
 'capital-loss': np.float64(0.020068855964263464),
 'education-num': np.float64(0.07095473385227195)}

## 3  Training & post-processing experiment

For each of 10 random splits we:

1. Fit a depth-5 Random-Forest

2. Run **Origin**, **Barycentre**, and **Partial** repairs

3. (Skip ROC here for brevity - see `02_adult.ipynb` for ROC details)

4. Record DI, F1 (macro/micro/weighted), and TV distance on our two repaired axes

In [6]:
thresh=0.05
x_list = ['age','education-num']
methods=['origin','barycentre','partial']
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)
    
    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label,linspace_range=(0.1,0.9),theta=1e-3)
    for method in methods[:-1]:
        report = pd.concat([report,projpost.postprocess(method)], ignore_index=True)
    
    for p in [1e-2,1e-3,1e-4]:
        report = pd.concat([report,projpost.postprocess('partial',para=p)], ignore_index=True)

report.to_csv('../data/E3_postprocess_adult_'+str(pa)+'.csv',index=None)

Let's aggregate across folds. In the table below:

- **DI**: Disparate Impact

- **F1 (macro/micro/weighted)**: classification quality

- **TV distance**: remaining gap on the repaired features

- **method**: repair strategy

In [7]:
report.dropna()

,DI,f1 macro,f1 micro,f1 weighted,TV distance,method
0,0.503565,0.670598,0.815683,0.784475,0.134117,origin
1,0.531465,0.670357,0.814096,0.783746,0.000968,barycentre
2,1.006899,0.597077,0.74556,0.724489,0.089893,partial_0.01
3,1.0728,0.591143,0.711522,0.706707,0.026697,partial_0.001
4,1.063186,0.594085,0.71362,0.708829,0.003719,partial_0.0001
5,0.46514,0.659247,0.814915,0.780162,0.134344,origin
6,0.495029,0.657153,0.810616,0.777578,0.000157,barycentre
7,1.077933,0.592332,0.728566,0.716057,0.089168,partial_0.01
8,1.109632,0.585663,0.702104,0.700979,0.026105,partial_0.001
9,0.800966,0.641019,0.766136,0.752284,0.003522,partial_0.0001


In [ ]:
thresh=0.3
x_list = ['age','education-num']
methods=['origin','barycentre','partial'] # Place ROC in the end
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(5):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label,linspace_range=(0.1,0.9),theta=1e-3)
    
    print(projpost.postprocess('origin'))
    print(projpost.postprocess('barycentre'))
    for t in range(2,4):
        print(projpost.postprocess('partial',para=10**(-t)))

# report.to_csv('../data/E3_postprocess_adult_'+str(pa)+'.csv',index=None)

         DI  f1 macro  f1 micro f1 weighted TV distance  method
0  0.594802  0.681166  0.814096     0.78887    0.132197  origin
         DI  f1 macro  f1 micro f1 weighted TV distance      method
0  0.609953  0.679836  0.814148    0.788323    0.000337  barycentre
         DI  f1 macro  f1 micro f1 weighted TV distance        method
0  0.641256  0.665039  0.808415    0.779688    0.087032  partial_0.01
         DI  f1 macro  f1 micro f1 weighted TV distance         method
0  0.644743  0.665263  0.808005     0.77962    0.028516  partial_0.001
         DI  f1 macro  f1 micro f1 weighted TV distance  method
0  0.462453  0.655602  0.812817    0.776241    0.137725  origin
         DI  f1 macro  f1 micro f1 weighted TV distance      method
0  0.517109  0.644492  0.809643    0.770117    0.000892  barycentre
         DI  f1 macro  f1 micro f1 weighted TV distance        method
0  0.520792  0.637826  0.807289    0.766267    0.092103  partial_0.01
         DI  f1 macro  f1 micro f1 weighted TV dis

In [9]:
valpost = Projpostprocess(X_val,y_val,x_list,var_list,clf,K,e,'auto',linspace_range=(0.1,0.9),theta=1e-3)
valpost.thresh

Optional threshold =  [0.1        0.18888889 0.27777778 0.36666667 0.45555556 0.54444444
 0.63333333 0.72222222 0.81111111 0.9       ]
Disparate Impact =  [0.71453015 0.6025167  0.60003295 0.59329543 0.59517592 0.59050674
 0.59051694 0.59993526 0.6137677  0.61607077]
f1 scores =  [0.69766103 0.66097452 0.65368215 0.65396777 0.65378049 0.65262052
 0.64995446 0.63836665 0.63192791 0.63129615]


np.float64(0.1)

### Analyzing results

- **Origin** buys the highest F1 but sits far below fairness (DI ≈ 0.50).

- **Barycentre** nearly closes the gap (TV ≈ 0) at only a tiny F1 penalty.

- **Partial** gives you a "fairness knob" - small t -> light fix, large t -> full parity at more cost

In [11]:
methods=['origin','unconstrained','barycentre','partial','ROC'] # Place ROC in the end
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label,linspace_range=(0.1,0.9),theta=1e-3)
    for method in methods[:-1]:
        # report = pd.concat([report,projpost.postprocess(method,para=1e-2)], ignore_index=True)
        report = pd.concat([report,projpost.postprocess(method,para=1e-3)], ignore_index=True)

    ROCpost = ROCpostprocess(X_val,y_val,var_list,clf,favorable_label) # use validation set to train a ROC model
    report = pd.concat([report,ROCpost.postprocess(X_test,y_test,tv_origin=projpost.tv_origin)], ignore_index=True)

report.to_csv('../data/report_postprocess_adult_'+str(pa)+'.csv',index=None)

Optimal classification threshold (with fairness constraints) = 0.1700
Optimal ROC margin = 0.0189
Optimal classification threshold (with fairness constraints) = 0.3100
Optimal ROC margin = 0.0344
Optimal classification threshold (with fairness constraints) = 0.1700
Optimal ROC margin = 0.0378
Optimal classification threshold (with fairness constraints) = 0.2300
Optimal ROC margin = 0.0256
Optimal classification threshold (with fairness constraints) = 0.1900
Optimal ROC margin = 0.0211
Optimal classification threshold (with fairness constraints) = 0.1900
Optimal ROC margin = 0.0211
Optimal classification threshold (with fairness constraints) = 0.1900
Optimal ROC margin = 0.0211
Optimal classification threshold (with fairness constraints) = 0.2900
Optimal ROC margin = 0.0322
Optimal classification threshold (with fairness constraints) = 0.1900
Optimal ROC margin = 0.0211
Optimal classification threshold (with fairness constraints) = 0.1900
Optimal ROC margin = 0.0211


## 4  Compute average feature importance

Finally, we revisit our Random-Forest baseline to see which inputs drive income prediction most:

* **capital-gain** and **education-num** lead the pack  

* **age** follows closely  

These are the features you’d want to guard most carefully against proxying sensitive traits.

In [12]:
pa = 'race'
label_map = {1.0: '>50K', 0.0: '<=50K'}
privileged_groups = [{pa: 1}]
unprivileged_groups = [{pa: 0}]
if pa == 'sex':
    thresh=0.05
    protected_attribute_maps = [{1.0: 'Male', 0.0: 'Female'}]
    cd = AdultDataset(protected_attribute_names=[pa],privileged_classes=[['Male'],[1.0]], 
        metadata={'label_map': label_map,'protected_attribute_maps': protected_attribute_maps},
        # categorical_features=['workclass', 'marital-status', 'occupation', 'relationship', 'native-country']
        features_to_drop=['race','fnlwgt','education','relationship',
                          'native-country','workclass','marital-status','occupation'])
elif pa == 'race':
    thresh=0.1
    protected_attribute_maps = [{1.0: 'White', 0.0:'Non-white'}]
    cd = AdultDataset(protected_attribute_names=[pa],privileged_classes=[['White'],[1.0]],
        metadata={'label_map': label_map,'protected_attribute_maps': protected_attribute_maps}, #
        features_to_drop=['sex','fnlwgt','education','relationship',
                          'native-country','workclass','marital-status','occupation'])
    #,'workclass','marital-status','occupation','relationship',

# train,test = cd.split([0.6], shuffle=True) #len(test.instance_names) = 2057
var_list = cd.feature_names.copy()
var_list.remove(pa)
var_dim=len(var_list)

K=200
e=0.01
bins_capitalgain=[100,3500,7500,10000]
bins_capitalloss=[100,1600,1900,2200]

messydata=cd.convert_to_dataframe()[0]
messydata=messydata.rename(columns={pa:'S',cd.label_names[0]:'Y'})
messydata=messydata[(messydata['S']==1)|(messydata['S']==0)]
for col in var_list+['S','Y']:
    messydata[col]=messydata[col].astype('int64')
messydata['W']=cd.instance_weights
# project 0-100 to {0,1,...,5}
messydata['age']=np.floor((messydata['age'].to_numpy()-17)/15)
messydata['hours-per-week']=np.floor(messydata['hours-per-week'].to_numpy()/20)
messydata=categerise(messydata,'capital-gain',bins_capitalgain)
messydata=categerise(messydata,'capital-loss',bins_capitalloss)

X=messydata[var_list+['S','W']].to_numpy() # [X,S,W]
y=messydata['Y'].to_numpy() #[Y]
tv_dist=dict()
for x_name in var_list:
    x_range_single=list(pd.pivot_table(messydata,index=x_name,values=['W'])[('W')].index) 
    dist=rdata_analysis(messydata,x_range_single,x_name)
    tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
x_list=[]
for key,val in tv_dist.items():
    if val>0.045:
        x_list+=[key]        
tv_dist

{'age': np.float64(0.04744857663969924),
 'education-num': np.float64(0.056762405582129784),
 'capital-gain': np.float64(0.021127950774052707),
 'capital-loss': np.float64(0.011363681253224836),
 'hours-per-week': np.float64(0.04445283428803036)}

## 5  Conclusions

In this notebook we compared several simple, model-agnostic post-processing strategies on a Random-Forest income predictor. 
Here’s what we found:

- **Post-processing is practical**: you can retrofit fairness without re-training.

- **Barycentre is powerful**: it erases group gaps with minimal accuracy loss.

- **Partial repair offers flexibility**: tune t to hit your compliance bar.
